<a href="https://colab.research.google.com/github/juliosdutra/Controle_de_Processos_UFES/blob/main/2.Model-linearization/Linear_reator_bioquimico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Instalar o pacote python-control.


In [1]:
! pip install -qq control

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.6/549.6 kB 11.0 MB/s eta 0:00:00


# 2. Importar o pacote control e demais bibliotecas.

In [2]:
import numpy as np
from scipy import linalg
import control as ct
from control import linearize

# 3. Implementar o modelo do processo e de observação.


In [3]:
def biorreator(t,x,u, params={}):

    '''modelo de um biorreator operando em batelada alimentada.'''

    # parametros do modelo
    mu_max = 0.20
    KS = 1.0
    Yxs = 0.5
    Ypx = 0.2

    # variáveis de estado
    X, P, S, V = x

    # variáveis de entrada
    Sf, F = u

    # equações constitutivas
    mu = mu_max*S/(KS+S)
    rg = mu*X
    rp = Ypx*rg

    # equações do modelo
    dV = F
    dX = (V*rg - dV*X)/V
    dP = (V*rp-dV*P)/V
    dS = (F*Sf - V*rg/Yxs - dV*S)/V

    # saida do modelo
    return [dX,dP,dS,dV]

def medida(t, x, u, params={}):

    '''Função de medida'''
    return x

# 4. Instanciar o modelo implementado.


In [4]:
# Instanciar o sistema I/O
reator = ct.NonlinearIOSystem(
            biorreator,
            outfcn = medida,
            states = ('X','P', 'S', 'V'),
            inputs = ('Sf','F'),
            outputs = ('X','P', 'S', 'V'))

# 5. Linearizar o modelo instanciado.

In [8]:
# Variáveis de entrada
Sf0 = 10.0 # Substrato na alimentação
F0 = 0.05 # Vazão de alimentação

# Variáveis de estado
X0 = 0.05
S0 = 10.0
P0 = 0.0
V0 = 1.0

# condição inicial (ponto estacionário)
xs = [X0, P0, S0, V0]
us = [Sf0, F0]

reator_linear = linearize(reator, xeq=xs, ueq=us)

StateSpace(array([[ 1.31818182e-01,  0.00000000e+00,  8.26446203e-05,
         2.49999750e-03],
       [ 3.63636364e-02, -5.00000000e-02,  1.65289241e-05,
         0.00000000e+00],
       [-3.63636364e-01,  0.00000000e+00, -5.01652893e-02,
         1.73472348e-11],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00]]), array([[ 0.  , -0.05],
       [ 0.  ,  0.  ],
       [ 0.05,  0.  ],
       [ 0.  ,  1.  ]]), array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]), array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]]))

# 6. Representar como espaços de estados.

In [10]:
# modelo linear
A = reator_linear.A
B = reator_linear.B
C = reator_linear.C
D = reator_linear.D

sys = ct.ss(A,B,C,D)
sys

StateSpace(array([[ 1.31818182e-01,  0.00000000e+00,  8.26446203e-05,
         2.49999750e-03],
       [ 3.63636364e-02, -5.00000000e-02,  1.65289241e-05,
         0.00000000e+00],
       [-3.63636364e-01,  0.00000000e+00, -5.01652893e-02,
         1.73472348e-11],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00]]), array([[ 0.  , -0.05],
       [ 0.  ,  0.  ],
       [ 0.05,  0.  ],
       [ 0.  ,  1.  ]]), array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]), array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]]))

# 7. Checar estabilidade

In [7]:
p, v = linalg.eig(A)

print('\n autovalores = \n', p)
print('\n autovetores = \n', v)

if any(np.real(p)>0):
    print('\n O sistema é INSTÁVEL.')
else:
    print('\n O sistema é ESTÁVEL.')


 autovalores = 
 [-0.05      +0.j  0.13165289+0.j -0.05      +0.j  0.        +0.j]

 autovetores = 
 [[ 0.00000000e+00  4.45435403e-01 -4.54545363e-04 -1.88678294e-02]
 [ 1.00000000e+00  8.90870806e-02 -9.72664330e-05 -1.36768450e-02]
 [ 0.00000000e+00 -8.90870806e-01  9.99999892e-01  1.36768450e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  9.90328905e-01]]

 O sistema é INSTÁVEL.
